In [1]:
! pip install "magentic[anthropic]"
! pip install dspy-ai PyPDF2


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 7.4 MB/s eta 0:00:00

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [2]:
!pwd

/Users/jaidhyani/Desktop/atefar


In [15]:
! pip uninstall -y atefar
! pip install -e .

Found existing installation: atefar 0.1.0
Uninstalling atefar-0.1.0:
  Successfully uninstalled atefar-0.1.0
Obtaining file:///Users/jaidhyani/Desktop/atefar
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Installing backend dependencies ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for atefar (pyproject.toml) ... done
  Created wheel for atefar: filename=atefar-0.1.0-py2.py3-none-any.whl size=895 sha256=8608e0a9242dc90429748827ec12d87c93c52c602affb7a176e1a04597151e0a
  Stored in directory: /private/var/folders/5k/7nfpl0cs5999pzhndyybcn800000gn/T/pip-ephem-wheel-cache-v66pt9bs/wheels/ef/a2/67/4e2ac8e515272dc3b0b25ecb48e15da2a9a7bc08f6adb925d8
Successfully built atefar


In [4]:
# update requirements.txt
! pip freeze > requirements.txt

In [5]:
! pip uninstall src

In [14]:
!pip list

Package            Version     Editable project location
------------------ ----------- -------------------------------
aiohappyeyeballs   2.3.5
aiohttp            3.10.2
aiosignal          1.3.1
alembic            1.13.2
annotated-types    0.7.0
anthropic          0.32.0
anyio              4.4.0
appnope            0.1.4
asttokens          2.4.1
atefar             0.1.0       /Users/jaidhyani/Desktop/atefar
attrs              24.2.0
backoff            2.2.1
certifi            2024.7.4
charset-normalizer 3.3.2
colorlog           6.8.2
comm               0.2.2
datasets           2.20.0
debugpy            1.8.5
decorator          5.1.1
dill               0.3.8
distro             1.9.0
dspy-ai            2.4.13
executing          2.0.1
filelock           3.15.4
filetype           1.2.0
frozenlist         1.4.1
fsspec             2024.5.0
h11                0.14.0
httpcore           1.0.5
httpx              0.27.0
huggingface-hub    0.24.5
idna               3.7
ipykernel          6.29.5
ip

In [9]:
import sys
print(sys.path) 

['/Users/jaidhyani/miniforge3/envs/mp4/lib/python311.zip', '/Users/jaidhyani/miniforge3/envs/mp4/lib/python3.11', '/Users/jaidhyani/miniforge3/envs/mp4/lib/python3.11/lib-dynload', '', '/Users/jaidhyani/Desktop/atefar/.venv/lib/python3.11/site-packages']


In [1]:
import dspy

import atefar
from atefar import paper_analysis, pdf_utils

/Users/jaidhyani/Desktop/atefar/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [44]:
from dataclasses import dataclass
from unittest import result
from dspy.primitives.program import Module
from typing import Any, Optional

from regex import F

@dataclass
class SigData:
    name: str
    desc: str

@dataclass
class SigStep:
    outputs: list[SigData]
    module: Module
    kwargs: Optional[dict[str, Any]] = None

@dataclass
class SigChain:
    inputs: list[SigData]
    steps: list[SigStep]

paper_chain = SigChain(
    inputs=[SigData("paper_content", "The full text content of the research paper")],
    steps=[
        SigStep(
            outputs=[
                SigData("title", "The title of the paper"),
                SigData("summary", "A concise summary of the paper's key ideas, target metrics, and strategies employed to target those metrics, with an eye towards reproducing the results"),
            ],
            module=dspy.ChainOfThought,
        ),
        SigStep(
            outputs=[
                SigData("core_ideas", "An enumerated list of the specific ideas or concepts that are central to the paper's main contributions"),
            ],
            module=dspy.ChainOfThought,
        ),
        SigStep(
            outputs=[
                SigData("metrics", "An enumerated list of the key metrics used in the paper. In particular, these should be objectively measurably quantities that measure the performance of the proposed methods."),
            ],
            module=dspy.ChainOfThought,
        ),
        SigStep(
            outputs=[
                SigData("methods", "An enumerated list of the key methods or strategies proposed in the paper to optimize the target metrics"),
            ],
            module=dspy.ChainOfThought,
        ),
        SigStep(
            outputs=[
                SigData("method_metric_results", "For each method, list the metric results that were reported in the paper. These should be the results that were used to compare the methods to each other."),
            ],
            module=dspy.ChainOfThought,
        ),
        SigStep(
            outputs=[
                SigData("hw_agnostic_metrics", "Metrics that correspond to the paper's metrics, but are not hardware-specific. These may be copied from input metrics where appropriate, or they may be generated if not otherwise available."),
            ],
            module=dspy.ChainOfThought,
        ),
        SigStep(
            outputs=[
                SigData(
                    "tasks", 
                    """
                    Describe each enumerated method as a function to be implemented. 
                    Describe what the inputs are and their types, what the outputs are and their types, 
                    and what the method does.  Do not describe how the function is implemented. It should 
                    be possible to verify the correctness of the implementation by comparing the output of the 
                    implementation to the output of the method din the paper.
                    """
                ),
            ],
            module=dspy.ChainOfThought,
        ),
        SigStep(
            outputs=[
                SigData(
                    "setup", 
                    """
                    A description of the setup required before implementing the task functions. This should include a list of assets that need to be available to start.
                    These may include datasets, trained models, training functions, etc. Assume that a python torch environment with GPUs is available. Any common libraries
                    or public assets may be used (e.g. huggingface transformers, torchvision, etc.). If a dataset is required, it should be available in a public location.
                    """
                ),
            ],
            module=dspy.ChainOfThought,
        ),
    ],
)

"""
    paper_content = dspy.InputField(desc="The full text content of the research paper")

    title = dspy.OutputField(desc="The title of the paper")
    summary = dspy.OutputField(desc="A concise summary of the paper's main contributions")
    core_ideas = dspy.OutputField(desc="The core idea(s) of the paper")
    methods = dspy.OutputField(desc="Key methods or strategies proposed in the paper")
    metrics = dspy.OutputField(desc="Primary metrics or evaluation criteria used in the paper")
    requirements = dspy.OutputField(desc="Key requirements for implementing and evaluating the paper's methods")
"""

def solve_sigchain(chain: SigChain, inputs=dict[str, str]) -> tuple[dict[str, Any], dict[str, Any]]:
    input_sigs = chain.inputs.copy()
    input_vals = inputs.copy()
    solvers = dict()
    results = dict()
    for step in chain.steps:
        step_name = "_".join([o.name for o in step.outputs])
        step_inputs_str = ", ".join([i.name for i in input_sigs])
        step_outputs_str = ", ".join([o.name for o in step.outputs])
        print(f"Running step {step_name}")
        print(f"  Inputs: {step_inputs_str}")
        print(f"  Outputs: {step_outputs_str}")
        class NextSig(dspy.Signature):
            pass
        for i in input_sigs:
            NextSig = NextSig.append(i.name, dspy.InputField(desc=i.desc))
        for o in step.outputs:
            NextSig = NextSig.append(o.name, dspy.OutputField(desc=o.desc))
        step_solver = step.module(NextSig)
        result = step_solver(**input_vals)
        for o in step.outputs:
            input_vals[o.name] = result[o.name]
        input_sigs.extend(step.outputs)
        solvers[step_name] = step_solver
        results[step_name] = result
    return solvers, results


def cascading_signatures(sig_elements: list[list[SigData]]) -> list[dspy.Signature]:
    inputs = sig_elements.pop(0)
    names = [i.name for i in inputs]
    assert len(set(names)) == len(names), "Input names must be unique"
    sigs = []
    while len(sig_elements) > 0:
        outputs = sig_elements.pop(0)
        class NextSig(dspy.Signature):
            pass
        for i in inputs:
            setattr(NextSig, i.name, dspy.Input(i.desc))
        for o in outputs:
            setattr(NextSig, o.name, dspy.Output(o.desc))
        sigs.append(NextSig)
        inputs.extend(outputs)

In [34]:
pdf_text = pdf_utils.extract_text_from_pdf("papers/94cifar.pdf")

In [45]:
solvers, results = solve_sigchain(paper_chain, {"paper_content": pdf_text})

Running step title_summary
  Inputs: paper_content
  Outputs: title, summary
Running step core_ideas
  Inputs: paper_content, title, summary
  Outputs: core_ideas
Running step metrics
  Inputs: paper_content, title, summary, core_ideas
  Outputs: metrics
Running step methods
  Inputs: paper_content, title, summary, core_ideas, metrics
  Outputs: methods
Running step method_metric_results
  Inputs: paper_content, title, summary, core_ideas, metrics, methods
  Outputs: method_metric_results
Running step hw_agnostic_metrics
  Inputs: paper_content, title, summary, core_ideas, metrics, methods, method_metric_results
  Outputs: hw_agnostic_metrics
Running step tasks
  Inputs: paper_content, title, summary, core_ideas, metrics, methods, method_metric_results, hw_agnostic_metrics
  Outputs: tasks
Running step setup
  Inputs: paper_content, title, summary, core_ideas, metrics, methods, method_metric_results, hw_agnostic_metrics, tasks
  Outputs: setup


In [49]:
print(results['tasks']['tasks'])

1. implement_vgg_like_network(widths, batchnorm_momentum)
   Inputs: 
   - widths: dictionary specifying the number of channels for each block
   - batchnorm_momentum: float for BatchNorm momentum
   Outputs: 
   - model: PyTorch nn.Module representing the network
   Description: Implements the modified VGG-like network architecture described in the paper.

2. initialize_whitening_conv(layer, train_set, eps)
   Inputs:
   - layer: first convolutional layer of the network
   - train_set: tensor of training images
   - eps: small constant for numerical stability
   Outputs:
   - None (modifies layer in-place)
   Description: Initializes the first convolutional layer as a patch-whitening transformation.

3. initialize_identity_conv(layer)
   Inputs:
   - layer: convolutional layer to initialize
   Outputs:
   - None (modifies layer in-place)
   Description: Initializes convolutional layers after the first as partial identity transforms.

4. alternating_flip(inputs, indices, epoch)
   Inpu

In [ ]:
class PaperAnalysis(dspy.Signature):
    """Analyze a research paper and provide core information."""

    paper_content = dspy.InputField(desc="The full text content of the research paper")

    title = dspy.OutputField(desc="The title of the paper")
    summary = dspy.OutputField(desc="A concise summary of the paper's main contributions")
    core_ideas = dspy.OutputField(desc="The core idea(s) of the paper")
    methods = dspy.OutputField(desc="Key methods or strategies proposed in the paper")
    metrics = dspy.OutputField(desc="Primary metrics or evaluation criteria used in the paper")
    requirements = dspy.OutputField(desc="Key requirements for implementing and evaluating the paper's methods")

In [14]:
def configure_lm():
    """Configure and return a language model."""
    lm = dspy.Claude("claude-3-5-sonnet-20240620", api_key="***REMOVED***")
    dspy.configure(lm=lm)
    return lm

def analyze_paper(pdf_path, model_name="gpt-3.5-turbo"):
    """Analyze a research paper given its PDF path."""

    # Configure LM
    lm = configure_lm()

    # Extract text from PDF
    paper_content = pdf_utils.extract_text_from_pdf(pdf_path)
 
    # Initialize and compile the analyzer
    analyzer = dspy.ChainOfThought(paper_analysis.PaperAnalysis)
    # Analyze the paper
    result = analyzer(paper_content=paper_content)

    return result

In [15]:
result=analyze_paper("papers/94cifar.pdf")

In [22]:
result.keys()

['rationale', 'summary', 'core_ideas', 'methods', 'metrics', 'requirements']

In [26]:
result.summary

str

In [24]:
result.methods

'1. Modified network architecture based on prior work, with some adjustments to layer sizes and initialization.\n2. Frozen patch-whitening initialization for the first convolutional layer.\n3. Identity initialization for subsequent convolutional layers.\n4. Optimization tricks including increased learning rate for BatchNorm biases and Lookahead optimization.\n5. Multi-crop evaluation using six augmented views of each test image.\n6. Alternating flip augmentation technique.\n7. Compilation using torch.compile for improved GPU utilization.'

In [25]:
result.metrics

'1. Training time to reach target accuracy (94%, 95%, or 96%)\n2. Number of FLOPs (Floating Point Operations) required\n3. Test set accuracy\n4. Class-aggregated calibration error (CACE)'